In [ ]:
import numpy as np
from utils import *

In [ ]:
n = 64 # Number of points in first point cloud
m = n # Number of points in second point cloud

# Generate point clouds a and b
gauss = lambda q, a, c: a * np.random.randn(2, q) + np.transpose(np.tile(c, (q, 1)))
a = (np.random.randn(2, n)*.3).transpose()
b = (np.hstack((gauss(int(m/2), .5, [0, 1.6]),
     np.hstack((gauss(int(m/4), .3, [-1, -1]), 
                gauss(int(m/4), .3, [1, -1])))))).transpose()

cloud_plot(a, b)